In [ ]:
# Install Required Packages
!pip install -q langchain langsmith openai python-dotenv
!pip install -q langchain-community langchain-openai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
# Load API Keys from Colab Secrets or Fallback
import os
from google.colab import userdata

try:
    os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGSMITH_API_KEY')
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    os.environ["LANGCHAIN_PROJECT"] = userdata.get('LANGSMITH_PROJECT')
    print(" API keys loaded from Colab secrets")
except:
    os.environ["LANGCHAIN_API_KEY"] = "your_langsmith_api_key_here"
    os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"
    os.environ["LANGCHAIN_PROJECT"] = "your_project_name_here"
    print(" Using direct API keys - consider using Colab secrets")

os.environ["LANGCHAIN_TRACING_V2"] = "true"


 API keys loaded from Colab secrets


In [ ]:
# Import Libraries and Setup LangSmith Project
import os, time, random, threading
from typing import Dict, Any, List

from langsmith import traceable
from langsmith.wrappers import wrap_openai
from openai import OpenAI

os.environ.setdefault("LANGSMITH_PROJECT", os.getenv("LANGCHAIN_PROJECT", "pr-dependable-headquarters-71"))

openai_client = wrap_openai(OpenAI())


In [ ]:
# Define Tool with Tracing
@traceable(run_type="tool", name="web_search_tool")
def web_search(query: str) -> List[str]:
    time.sleep(0.05 + random.uniform(0.0, 0.05))
    return [f"Result A for {query}", f"Result B for {query}"]


In [ ]:
# Define Agent Class with Tracing
class SingleAgent:
    def __init__(self, name="agent-1", speed=1.0):
        self.name = name
        self.speed = speed
        self.metrics = {"done": 0, "errors": 0, "latencies": []}
        self.lock = threading.Lock()

    @traceable(run_type="chain", name="SingleAgent.run")
    def run(self, task: Dict[str, Any]) -> Dict[str, Any]:
        start = time.time()

        docs = web_search(task["query"])

        system_msg = "Answer using only provided results. Be concise."
        user_q = f"Q: {task['query']}\nResults:\n- " + "\n- ".join(docs)
        resp = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_msg},
                {"role": "user", "content": user_q},
            ],
            temperature=0.2,
        )

        time.sleep((0.05 + random.uniform(0, 0.05)) / self.speed)

        ok = random.random() > 0.05
        latency = time.time() - start
        with self.lock:
            if ok:
                self.metrics["done"] += 1
            else:
                self.metrics["errors"] += 1
            self.metrics["latencies"].append(latency)

        output_content = ""
        if resp and resp.choices:
            output_content = resp.choices[0].message.content

        return {
            "agent": self.name,
            "ok": ok,
            "latency_s": round(latency, 3),
            "output": output_content,
        }


In [ ]:
# Run Agent on Tasks and Print Full Output
if __name__ == "__main__":
    agent = SingleAgent()
    tasks = [
        {"id": i, "query": q}
        for i, q in enumerate([
            "Best South Indian breakfast options in Bengaluru",
            "How to implement a thread-pooled agent orchestrator in Python",
            "Trade-offs of vertical vs horizontal scaling for LLM agents",
            "Simple KPIs to monitor an AI agent in production"
        ])
    ]

    results = []
    for t in tasks:
        out = agent.run(t)
        print(f"\nTask {t['id']} - ok={out['ok']} latency={out['latency_s']}s")
        print(f"Output:\n{out['output']}")

    print("\n Done. Open LangSmith project to inspect:")
    print(f"Project: {os.getenv('LANGCHAIN_PROJECT', 'default')}")
    print("Runs include: SingleAgent.run (parent), web_search_tool (child), OpenAI chat (child).")



Task 0 - ok=True latency=0.956s
Output:
The best South Indian breakfast options in Bengaluru include those highlighted in Result A and Result B.

Task 1 - ok=True latency=12.186s
Output:
To implement a thread-pooled agent orchestrator in Python, you can use the `concurrent.futures.ThreadPoolExecutor` class. Here’s a concise outline:

1. **Import Required Modules**:
   ```python
   from concurrent.futures import ThreadPoolExecutor, as_completed
   ```

2. **Define Your Agent Function**:
   Create a function that represents the task each agent will perform.
   ```python
   def agent_task(agent_id):
       # Perform task
       return f"Agent {agent_id} completed"
   ```

3. **Set Up the Thread Pool**:
   Use `ThreadPoolExecutor` to manage the pool of threads.
   ```python
   def orchestrator(num_agents):
       with ThreadPoolExecutor(max_workers=num_agents) as executor:
           futures = {executor.submit(agent_task, i): i for i in range(num_agents)}
           for future in as_compl

# 🔍 LangSmith AI Agent Observability Demo

A comprehensive demonstration of AI agent monitoring and tracing using LangSmith, featuring a single agent with OpenAI integration and tool calling capabilities for production-grade observability.

## 🌟 Features

- **📊 Complete Observability**: Full tracing of agent execution with LangSmith integration
- **🔧 Tool Integration**: Simulated web search tool with performance tracking
- **🤖 OpenAI Integration**: GPT-4o-mini model with automatic trace capture
- **📈 Performance Metrics**: Built-in latency, success rate, and error tracking
- **🔒 Secure Key Management**: Google Colab secrets integration with fallback options
- **🎯 Real-world Tasks**: Practical examples including Bengaluru food recommendations and AI development

## 🛠️ Installation & Setup

### Prerequisites

The demo requires several Python packages for LangChain, LangSmith, and OpenAI integration:
- langchain & langsmith for agent orchestration and tracing
- openai for LLM capabilities
- python-dotenv for environment management
- langchain-community & langchain-openai for extended functionality

### API Keys Required

You'll need two API keys:
1. **LangSmith API Key**: Get from LangSmith settings dashboard
2. **OpenAI API Key**: Get from OpenAI platform

### Environment Configuration

The demo supports multiple configuration methods:

**Option 1: Google Colab Secrets (Recommended)**
- Store API keys securely in Colab's secret management system
- Keys are automatically loaded without exposure in code

**Option 2: Direct Environment Variables**
- Set LANGSMITH_API_KEY and OPENAI_API_KEY directly
- Less secure but useful for local development

**Option 3: Direct Assignment**
- Fallback method for testing (not recommended for production)

## 🏗️ Architecture Overview

### Component Structure

**SingleAgent Class**
- Main agent orchestrator with built-in performance tracking
- Thread-safe metrics collection
- Configurable execution speed for testing different scenarios

**Tool Integration**
- Web search tool simulation with realistic latency
- Automatic tracing as child runs in LangSmith
- Configurable response patterns

**OpenAI Integration**
- Wrapped OpenAI client for automatic trace capture
- Uses GPT-4o-mini for cost-effective processing
- System prompts optimized for concise, relevant responses

### Tracing Hierarchy

The system creates a hierarchical trace structure:
1. **Parent Run**: SingleAgent.run (main agent execution)
2. **Child Run 1**: web_search_tool (tool execution)
3. **Child Run 2**: OpenAI chat completion (LLM call)

## 🚀 Demo Workflow

### Task Execution Process

1. **Tool Invocation**: Agent calls web search tool with user query
2. **Result Processing**: Tool returns simulated search results
3. **LLM Integration**: Results fed to GPT-4o-mini for answer generation
4. **Response Synthesis**: Agent combines tool output with LLM response
5. **Metrics Collection**: Performance data captured for analysis

### Sample Tasks

The demo includes real-world query examples:
- **Local Knowledge**: "Best South Indian breakfast options in Bengaluru"
- **Technical Implementation**: "How to implement a thread-pooled agent orchestrator in Python"
- **System Design**: "Trade-offs of vertical vs horizontal scaling for LLM agents"
- **Production Monitoring**: "Simple KPIs to monitor an AI agent in production"

## 📊 Monitoring & Observability

### LangSmith Integration

**Automatic Tracing**
- Every agent execution creates a complete trace
- Tool calls and LLM requests automatically captured as child runs
- Full request/response logging for debugging

**Performance Tracking**
- Real-time latency measurements
- Success/failure rate monitoring
- Error categorization and logging

**Visual Dashboard**
- LangSmith web interface shows execution flows
- Detailed run information including inputs, outputs, and timing
- Search and filter capabilities for specific runs or patterns

### Built-in Metrics

**Execution Metrics**
- Task completion count
- Error frequency and types
- Response time distributions

**Quality Metrics**
- Success rate tracking (95% target with 5% simulated failures)
- Output quality assessment through response analysis

## 🔧 Configuration Options

### Agent Parameters

**Speed Adjustment**
- Configurable execution speed multiplier for testing scenarios
- Useful for simulating different load conditions

**Error Simulation**
- Built-in 5% error rate for realistic testing
- Helps validate error handling and recovery mechanisms

### LangSmith Project Settings

**Project Organization**
- Default project: "agent-observability-demo"
- Configurable via environment variables
- Supports multiple project environments

**Tracing Configuration**
- Full tracing enabled by default
- Configurable trace levels and sampling rates

## 🎯 Use Cases

### Development & Testing

**Agent Development**
- Monitor agent behavior during development
- Identify performance bottlenecks
- Debug tool integration issues

**Load Testing**
- Simulate multiple concurrent agent executions
- Measure performance under different load conditions
- Optimize resource utilization

### Production Monitoring

**Real-time Observability**
- Monitor live agent performance
- Track success rates and response times
- Alert on anomalous behavior

**Quality Assurance**
- Verify agent responses meet quality standards
- Monitor tool effectiveness
- Track user satisfaction metrics

## 📈 Performance Analysis

### Key Metrics Dashboard

**Response Time Analysis**
- Average, min, max response times
- Response time distribution patterns
- Performance trends over time

**Success Rate Monitoring**
- Task completion rates
- Error categorization and frequency
- Recovery time analysis

**Resource Utilization**
- API call frequency and costs
- Tool usage patterns
- System resource consumption

### Optimization Insights

**Bottleneck Identification**
- Slowest components in execution chain
- Tool vs LLM performance comparison
- Network latency impact analysis

**Cost Optimization**
- API usage patterns and costs
- Token consumption analysis
- Efficiency improvement opportunities

## 🚨 Error Handling & Debugging

### Built-in Error Simulation

The demo includes realistic error scenarios:
- Random 5% failure rate for testing resilience
- Network timeout simulation
- API rate limiting scenarios

### Debugging Features

**Comprehensive Logging**
- Detailed execution logs for each step
- Error stack traces and context
- Performance timing breakdown

**LangSmith Integration**
- Visual trace inspection in web interface
- Failed run analysis and debugging
- Historical error pattern analysis

## 🔒 Security & Best Practices

### API Key Management

**Secure Storage**
- Google Colab secrets integration
- Environment variable protection
- No hardcoded credentials

**Access Control**
- Project-based access management
- API key rotation capabilities
- Audit trail for key usage

### Data Privacy

**Trace Data Protection**
- Configurable data retention policies
- PII scrubbing capabilities
- Compliance with data protection regulations

## 🚀 Deployment Considerations

### Local Development

**Setup Requirements**
- Python environment with required packages
- API key configuration
- LangSmith account setup

**Testing Configuration**
- Local project isolation
- Development vs production traces
- Mock tool integration for offline testing

### Production Deployment

**Scalability Planning**
- Multi-agent orchestration support
- Load balancing considerations
- Resource allocation optimization

**Monitoring Infrastructure**
- Integration with existing monitoring systems
- Custom alerting rules
- Performance baseline establishment

## 📚 Learning Outcomes

### Agent Development Skills

**Observability Implementation**
- Understanding trace hierarchy design
- Performance metric collection strategies
- Error handling and recovery patterns

**Production Readiness**
- Monitoring best practices for AI systems
- Quality assurance for agent responses
- Cost optimization techniques

### LangSmith Expertise

**Platform Utilization**
- Trace analysis and debugging
- Performance optimization insights
- Team collaboration features

**Integration Patterns**
- OpenAI wrapper usage
- Custom tool tracing
- Multi-component system monitoring

## 🤝 Next Steps & Extensions

### Enhancement Opportunities

**Multi-Agent Support**
- Extend to multiple concurrent agents
- Agent coordination and communication
- Distributed tracing across agent networks

**Advanced Tool Integration**
- Real API integrations (web search, databases)
- Custom tool development and tracing
- Tool performance optimization

**Production Features**
- Advanced error recovery mechanisms
- Dynamic configuration management
- Automated performance optimization

### Community Contributions

Areas for contribution and extension:
- Additional tool integrations
- Enhanced error simulation scenarios
- Performance optimization techniques
- Documentation improvements

## 📞 Support & Resources

### Getting Help

**LangSmith Documentation**
- Official platform documentation
- Best practices guides
- Community forums and support

**OpenAI Integration**
- API documentation and examples
- Cost optimization guides
- Model selection recommendations

### Community Resources

- Sample implementations and extensions
- Performance benchmarking results
- Integration pattern libraries

